In [1]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import networkx as nx
import mdtraj
import numpy as np
from dgllife.utils import mol_to_bigraph
import pickle
import random
from rdkit.Chem import rdDepictor
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFreeSASA
import torch
import time
import hashlib
from scipy.spatial import distance_matrix
from sqlalchemy import create_engine
import math
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
import _mysql
from rdkit.Chem import rdMolDescriptors
from sklearn.cluster import KMeans
from scipy.spatial import distance_matrix
import sklearn as sk
import scipy.stats
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import OneHotEncoder
import copy

In [2]:
###DB와 연결하는 Class
class CallDB:
    def __init__(self):
        self.engine = create_engine("mysql+mysqldb://{}:{}@{}/{}".format(USER_NAME, USER_PSWD, DB_HOST, DB_NAME), encoding='utf-8')
        
    def query_db(self, query, DB_NAME):
        dbconn = _mysql.connect(host=DB_HOST, user=USER_NAME, passwd=USER_PSWD, port=PORT, db=DB_NAME)
        self.cursor = dbconn.cursor()
        self.cursor.execute(query)
        dbconn.commit()
        dbconn.close()
    
    def to_db(self, data, table_nm):
        df = pd.DataFrame(data)
        df.to_sql(name=table_nm, con=self.engine, if_exists='append', index=False)
        
    def from_db(self, query,DB_NAME):
        dbconn = _mysql.connect(host=DB_HOST, user=USER_NAME, passwd=USER_PSWD, port=PORT, db=DB_NAME, cursorclass = pymysql.cursors.DictCursor)
        self.cursor = dbconn.cursor()
        self.cursor.execute(query)
        table = self.cursor.fetchall()
        dbconn.commit()
        dbconn.close()
        return pd.DataFrame(table)

In [3]:
##clustering 하는 함수
A1 = 0.8
A2 = 0.8
CLT_ERR_NUMS=[]


class ClusteringEnsemble:
    def __init__(self):
        self.a1 = 0.8
        self.a2 = 0.8        
        self.clt_result = pd.DataFrame({'km0':[], 'km1':[], 'sc0':[], 'sc1':[], 'ac':[]})

    def transformation(self, sample):
        km0 = KMeans(n_clusters=2, random_state=0).fit(sample)
        self.clt_result['km0'] = km0.labels_
        
        km1 = KMeans(n_clusters=2, random_state=111).fit(sample)
        self.clt_result['km1'] = km1.labels_
        
        sc0 = SpectralClustering(n_clusters=2,  random_state=0).fit(sample)
        self.clt_result['sc0'] = sc0.labels_
        
        sc1 = SpectralClustering(n_clusters=2,  random_state=111).fit(sample)
        self.clt_result['sc1'] = sc1.labels_
        
        ac = AgglomerativeClustering(n_clusters=2).fit(sample)
        self.clt_result['ac'] = ac.labels_
        
        enc = OneHotEncoder()
        enc.fit(np.array(self.clt_result['km0']).reshape(-1, 1))
        
        km0_onehot = enc.transform(np.array(self.clt_result['km0']).reshape(-1, 1)).toarray()
        km1_onehot = enc.transform(np.array(self.clt_result['km1']).reshape(-1, 1)).toarray()
        sc0_onehot = enc.transform(np.array(self.clt_result['sc0']).reshape(-1, 1)).toarray()
        sc1_onehot = enc.transform(np.array(self.clt_result['sc1']).reshape(-1, 1)).toarray()
        ac_onehot = enc.transform(np.array(self.clt_result['ac']).reshape(-1, 1)).toarray()
        
        onehot_cst = pd.DataFrame(np.hstack((km0_onehot, km1_onehot, sc0_onehot, sc1_onehot, ac_onehot)),
                                     columns=('km0_0', 'km0_1',
                                              'km1_0', 'km1_1',
                                              'sc0_0', 'sc0_1',
                                              'sc1_0', 'sc1_1',
                                              'ac_0', 'ac_1'))
        
        return onehot_cst
    
    
    def consensus_function(self, onehot_cst): 
        while len(onehot_cst.columns) > 2:    # replace constant!
            # calculate cluster similarity
            cluster_sim = onehot_cst.corr()    # Pearson's R, cluster similarity
            
            # cluster similarity matrix -> upper triangular matrix
            mask = (np.triu(np.ones(len(onehot_cst.columns))) - np.eye(len(onehot_cst.columns))).astype(np.bool)
            cluster_sim = cluster_sim.where(mask)
            cluster_sim = cluster_sim.stack().reset_index()
            cluster_sim.columns = ['cluster1', 'cluster2', 'similarity']
            
            # extract argmax cluster similarity
            if cluster_sim['similarity'].max() > A1:
                maxval_pair = cluster_sim.iloc[cluster_sim['similarity'].idxmax()]
            else:
                return onehot_cst
            
            # update cluster matrix
            new_cst = onehot_cst[maxval_pair['cluster1']] + onehot_cst[maxval_pair['cluster2']]
            onehot_cst[maxval_pair['cluster1'] + '+' + maxval_pair['cluster2']] = new_cst
            onehot_cst = onehot_cst.drop(columns=[maxval_pair['cluster1'], maxval_pair['cluster2']])
    
        return onehot_cst
    
    
    def membership_similarity(self, onehot_cst):
        membership_sim = onehot_cst / max(onehot_cst.max())
        return membership_sim
        
        
    def elimination(self, onehot_cst):
        membership_sim = self.membership_similarity(onehot_cst)
        cluster_qual = membership_sim[membership_sim!=0].mean()
        cluster_qual = cluster_qual.sort_values(ascending=False)[:3]
        membership_sim = membership_sim[list(cluster_qual.index)]
        return membership_sim
        
    
    def minimum_effect(self, membership_sim):
        cluster_var = membership_sim[membership_sim>A2].var()
        certain_mask = membership_sim >= A2
        
        determined_cst = certain_mask
        uncertain_list = membership_sim.max(axis=1) < A2
        uncertain_list = list(uncertain_list[uncertain_list==True].index)    
        for i in uncertain_list:
            candidate_test = certain_mask
            candidate_test.iloc[i, :] = True
            candidate_var = membership_sim[candidate_test].var()
            elected = abs(cluster_var - candidate_var).idxmin()
            determined_cst[elected][i] = True
        
        membership_sim = membership_sim[determined_cst].fillna(0)
        return membership_sim
        
    
    def enforce_clustering(self, membership_sim):
        membership_sim.columns = list(range(len(membership_sim.columns)))    # 컬럼이름 숫자로 바꾸기
        
        if (membership_sim.max(axis=1) > A2).all() == False:    # when all objects are not certain
            membership_sim = self.minimum_effect(membership_sim)
            
        last_cluster = membership_sim.idxmax(axis=1)
        
        return last_cluster

    def ensemble(self, sample):
        onehot_cst = self.transformation(sample)
        onehot_cst = self.consensus_function(onehot_cst)
        membership_sim = self.elimination(onehot_cst)
        last_cluster = self.enforce_clustering(membership_sim)
        
        return last_cluster
    
    

In [4]:
###DB에서 분석목록 불러오기
DB_HOST = "211.233.58.16"
PORT = 3306  #포트번호
USER_NAME = "jhjeong"
USER_PSWD = "93"
DB_NAME="dds_tmp"

db = CallDB()

query = "SELECT * FROM input_result WHERE rqnum=(SELECT rqnum FROM input_result order by rqnum desc limit 1);"
#query = "SELECT * FROM input_result WHERE rqnum='rq00006';"

dataset = db.from_db(query, DB_NAME)

In [5]:
##DB에서 불러온 데이터 Dictionary로 변환
smiles_input_dic = {}
for i, row in dataset.iterrows():
    smiles_input_dic[row['mdb_id']] = row['canonical_smiles']

ERR_NUMS = []

#Dictionary -> Tuple형태로 가공
class SmilesToInput:
    def __init__(self, smiles_input_dic, coords = {}, adj = {}, feat = {}, smiles = {}, pps = {}):
        self.smiles_input = smiles_input_dic
        self.coords_dic = coords
        self.adj_dic = adj
        self.feat_dic = feat
        self.smiles_dic = smiles
        self.pps = pps

    def make_dic(self):     
        def featurize_bonds(mol):
            feats = []
            bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE,
                          Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]
            for bond in mol.GetBonds():
                btype = bond_types.index(bond.GetBondType())
                # One bond between atom u and v corresponds to two edges (u, v) and (v, u)
                feats.extend([btype, btype])
            return {'type': torch.tensor(feats).reshape(-1, 1).float()}
        
        for idx, (key, value) in enumerate(self.smiles_input.items()):   # 각 molecule에 대해
            try:
                mol = Chem.MolFromSmiles(value)                 ## Mol to Smiles
                mol = Chem.RemoveHs(mol)
                mol = Chem.AddHs(mol)

                rdDepictor.Compute2DCoords(mol,canonOrient=False)
                AllChem.EmbedMultipleConfs(mol, 5)    # 원하는 수만큼 conformer pool 생성
                confs = AllChem.MMFFOptimizeMoleculeConfs(mol, maxIters=1200)
                engys = [c[1] for c in confs]
                min_e_idx = engys.index(min(engys))
                conf = mol.GetConformer(min_e_idx)

                x, y, z = [], [], []
                for k in range(mol.GetNumAtoms()):
                    pos = conf.GetAtomPosition(k)
                    x.append(pos.x); y.append(pos.y); z.append(pos.z)
                x = np.array(x); y = np.array(y); z = np.array(z)
                codi = np.vstack((x, y, z)).T
                self.coords_dic[key] = codi                     # 좌표 저장


                grp = mol_to_bigraph(mol, edge_featurizer=featurize_bonds, canonical_atom_order=False)
                grp_mat = grp.adjacency_matrix()
                grp_mat = np.array(grp_mat.to_dense())

                num_atoms = mol.GetNumAtoms()
                for i in range(num_atoms):
                    for j in range(num_atoms):
                        if grp.edge_id(i,j, return_array=1).size()[0] != 0:
                            grp_mat[j,i] += int(grp.edata['type'][int(grp.edge_id(i,j, return_array=1))])
                self.adj_dic[key] = grp_mat                      # 그래프 저장

                atoms = mol.GetAtoms()
                atomicnums = []
                for j, atom in enumerate(atoms):
                    atomicnums.append(atom.GetAtomicNum())
                self.feat_dic[key] = np.array(atomicnums)        # 원자번호 저장

                self.smiles_dic[key] = value                    # smiles 저장

                tpsa = Descriptors.TPSA(mol)
                radii = rdFreeSASA.classifyAtoms(mol)
                sasa = rdFreeSASA.CalcSASA(mol, radii)
                self.pps[key] = tpsa / sasa                     # pps 저장
                
            except:
                print(idx, key)
                ERR_NUMS.append(key)

        return self.adj_dic, self.feat_dic, self.coords_dic, self.smiles_dic, self.pps

In [6]:
def ion_too_much(adj, feat):
    too_much_list = []
    idx = []
    for i in range(len(adj)):
        mol_adj = np.array(adj[list(adj)[i]], dtype=np.int64) 
        mol_feat = feat[list(feat)[i]]
                                                                #여기 밑에 mol_adj 수정해야할듯
        ion_list = adj[list(adj)[i]].sum(axis=0)==0             #해당 분자 내의 이온 원자들
        if sum(ion_list) >= 2:                                  #이온 원자가 2개 이상일 경우     
            too_much_list.append(list(adj.keys())[i])           #too_much_list에 index 기록
        else:
            idx.append(list(adj.keys())[i])                     #이온이 한개미만인 원자만 분석 리스트로 추출
    return(too_much_list, idx)

In [7]:
def oh_ring(adj, feat, idx):
    oh_adj = np.matrix([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #1
                        [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #2
                        [0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #3
                        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #4
                        [0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #5
                        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #6
                        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #7
                        [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #8
                        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], #9
                        [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #10
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #11
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #12
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #13
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], #14
                        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0], #15
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], #16
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0], #17
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], #18
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0], #19
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], #20
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], #21
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], #22
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2], #23
                        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]])
    oh_feat = np.matrix([1,8,6,1,6,1,1,8,6,6,1,1,8,1,6,1,8,1,6,1,1,8,6,8]).T
    matmuled = np.matmul(oh_adj, oh_feat).A1
    matmuled =  sorted(matmuled)
    matmuled = ''.join(map(str, matmuled))
    
    #hash처리
    hash_object = hashlib.sha256()
    hash_object.update(matmuled.encode())    
    hex_dig1 = hash_object.hexdigest()    
    
    oh_ring_list = []
    
    for i in range(len(idx)):
        #mol to hash
        mol_adj = np.array(adj[list(adj)[i]], dtype=np.int64) 
        mol_feat = feat[list(feat)[i]]
        
        matmuled = np.matmul(mol_adj, mol_feat)
        matmuled =  sorted(matmuled)
        matmuled = ''.join(map(str, matmuled))
        
        hash_object = hashlib.sha256()
        hash_object.update(matmuled.encode())    
        hex_dig2 = hash_object.hexdigest() 
        
        #compare
        if hex_dig1 == hex_dig2 :
             oh_ring_list.append(list(adj.keys())[i])                 #oh_ring인 분자 기록
             idx.remove(list(adj.keys())[i])                #oh_ring 아닌 분자만 분석 리스트로 추출
        
    return(oh_ring_list, idx)

In [8]:
class calculating:
    def __init__(self, mol_adj, mol_feat):
        self.mol_adj = mol_adj
        self.mol_feat = mol_feat
        
        
    def findson_index(self, a, father=None):
        """ input(a)의 아들 노드 인덱스 추출 (father = list형태로 입력)"""
        index = list(np.where(self.mol_adj[a,]==1)[0]) + list(np.where(self.mol_adj[a,]==2)[0]) + list(np.where(self.mol_adj[a,]==4)[0]) #연결이 존재하는 인덱스들
       
        if father == None:
            pass
        else:
            index = list(set(index) - set(father))
        
        return index

   # def findson_atom(self, a, father=None):
   #     """ input(a)의 아들 노드의 원자 추출 (father = list형태로 입력) """
   #     index = list(np.where(self.mol_adj[a,]==1)[0]) + list(np.where(self.mol_adj[a,]==2)[0]) +list(np.where(self.mol_adj[a,]==4)[0])#연결이 존재하는 인덱스들
   #     
   #     if father == None:
   #         pass
   #     else:
   #         index = list(set(index) - set(father))
   #     
   #     atom = self.mol_feat[index]
   #     return atom
    
    def findson_atom(self, a, father=None):
        """ input(a)의 아들 노드의 원자 추출 (father = list형태로 입력) """
        index = list(np.where(self.mol_adj[a,]==1)[0]) + list(np.where(self.mol_adj[a,]==2)[0]) +list(np.where(self.mol_adj[a,]==4)[0])#연결이 존재하는 인덱    
        
        if father == None:
            atom = self.mol_feat[index]
        
        else:
            index = list(set(index) - set(father))    
            atom = self.mol_feat[index]
        
        return atom


   
    def correct_atom_num(self, universal, subset):
        """ subset중 universal에 속하는애들이 몇개인지 추출 """
        num=0
        universal = list(universal)
        for i in range(len(subset)):
            if subset[i] in universal :
                num=num+1
                del universal[universal.index(subset[i])]
        return num

    def get_multiple_elements(self, in_list, in_indices):
        """리스트에서 복수인덱스 값을 가져온다"""
        return [in_list[i] for i in in_indices]        
        
    
    def so4na(self):
        so4na_list = []
   
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i]==16:         #i번째 원소가 S일경우          
                if sorted(self.findson_atom(i)) == [6,6,6,6] :   #아들노드가 OOOO인가?
                    candidate1 = sorted(self.findson_index(i))
                    if(sum(self.mol_adj[candidate1,].sum(axis=1)!=2)==1 and sum(self.mol_adj[np.where(self.mol_feat==11),].sum(axis=2)[0]==0)==1): 
                        #아들중 다리 비정상 1개 존재 and Na 원자의 연결이 0인것이 1개 존재
                        
                        tmp1 = np.where(self.mol_feat==11)[0]
                        tmp2 = np.where(self.mol_adj.sum(axis=1)==0)[0]
                        candidate2 = list(tmp1 & tmp2)                           #해당 경우를 후보2로 추가

                        candidate1.extend([i])
                        candidate1.extend(candidate2)
                        so4na_list.append(sorted(candidate1)) 
                        
        del_list = sum(so4na_list, [])       #삭제해야 할 목록들       
    
       # self.mol_adj[del_list,:] = 0  
       # self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
                        
        return(so4na_list)
    
    
    def coona(self):
        coona_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 6 :   #i번째 원소가 C일 경우
                if self.correct_atom_num(self.findson_atom(i), [6,6]) >=2 :     #c의 아들중 o가 2개 이상일 경우
                    candidate1 = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==8)[0]) #cand1 : c 아들중 o인애들
                    if sorted(self.mol_adj[candidate1,].sum(axis=1)) == [1,2]:   #아들 o와 연결이 1중 1개, 2중 1개 존재
                        if sum(self.mol_adj[np.where(self.mol_feat==11),].sum(axis=2)[0]==0)==1: # na원자의 연결이 0인 경우가 1개일 경우
                            tmp1 = np.where(self.mol_feat==11)[0]
                            tmp2 = np.where(self.mol_adj.sum(axis=1)==0)[0]
                            candidate2 = list(tmp1 & tmp2)                           #해당 경우를 후보2로 추가
                            
                            candidate1.extend([i])
                            candidate1.extend(candidate2)
                            coona_list.append(sorted(candidate1)) 
                            
        del_list = sum(coona_list, [])       #삭제해야 할 목록들       
    
      #  self.mol_adj[del_list,:] = 0  
      #  self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
        
        return coona_list
    
    
    def cook(self):
        cook_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 6 :   #i번째 원소가 C일 경우
                if self.correct_atom_num(self.findson_atom(i), [6,6]) >=2 :     #c의 아들중 o가 2개 이상일 경우
                    candidate1 = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==8)[0]) #can1 : c 아들중 o인애들
                    if sorted(self.mol_adj[candidate1,].sum(axis=1)) == [1,2]:   #아들 o와 연결이 1중 1개, 2중 1개 존재
                        if sum(self.mol_adj[np.where(self.mol_feat==19),].sum(axis=2)[0]==0)==1: # k원자의 연결이 0인 경우가 1개일 경우
                            tmp1 = np.where(self.mol_feat==19)[0]
                            tmp2 = np.where(self.mol_adj.sum(axis=1)==0)[0]
                            candidate2 = list(tmp1 & tmp2)                           #해당 경우를 후보2로 추가
                            
                            candidate1.extend([i])
                            candidate1.extend(candidate2)
                            cook_list.append(sorted(candidate1)) 
                            
        del_list = sum(cook_list, [])       #삭제해야 할 목록들       
    
       # self.mol_adj[del_list,:] = 0  
       # self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
        
        return cook_list
    
    def nr2(self):
        nr2_list = []
        del_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 7 :      #i번째 원소가 N일 경우
                if len(self.findson_atom(i))==3 :       #i번째 원소의 아들이 3개일경우
                    candidate1 = self.findson_index(i)
                        
                    nr2_list.append(sorted([i] + candidate1))
                    del_list.extend([i])
            
      #  self.mol_adj[del_list,:] = 0  
      #  self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
        
        return(nr2_list) 
    
    def coo_ring(self):
        coo_ring_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 6 :    #i번째 원소가 C일 경우
                 if self.correct_atom_num(self.findson_atom(i), [6,6,1,8]) >=4 :     #c의 아들이 c,c,o,h인가?
                     candidate1 = self.findson_index(i)                         #cand1: 뿌리노드의 아들
                     next_step = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==6)[0]) #next_step:2세대중 C인애들(2개)
                     if self.correct_atom_num(self.findson_atom(next_step[0], father=[i]), [1,6,8]) ==3:
                         if self.correct_atom_num(self.findson_atom(next_step[1], father=[i]), [1,6,8]) ==3:                   #2세대 중 C인애들의 아들이 COH인가(2번)
                             
                             candidate2 = []
                             for j in range(len(next_step)):                             
                                 candidate2.extend(self.findson_index(next_step[j], father=[i]))                          #3세대 할당
                                                                            #3세대중 H제외
                             next_step = self.get_multiple_elements(candidate2, np.where(self.mol_feat[candidate2]!=1)[0])   
                             
                             #3세대 중 h가 아닌애들의 아들이 [och, hho, h, h]인가?
                             tmp_candidate3 = []
                             for j in range(len(next_step)):
                                 tmp = sorted(list(self.findson_atom(next_step[j], candidate1)))
                                 tmp_candidate3.append(tmp) 
                             if self.correct_atom_num(tmp_candidate3, [[1,6,8], [1,1,8], [1], [1]]) == 4 :   #tmp_cand3:4세대 후보 검증중
                                 candidate3 = []
                                 for j in range(len(next_step)):                                        #cand3 할당
                                     tmp = sorted(list(self.findson_index(next_step[j], [i] + candidate1 + candidate2)))
                                     candidate3.extend(tmp)
                                 
                                 #4세대 중 h가 아닌애들의 아들이 [h, hho, c]인가?
                                 #next_step:4세대 중 h가 아닌 놈들
                                 next_step = []
                                 for j in range(len(candidate2)):
                                     next_step.extend(self.get_multiple_elements(self.findson_index(candidate2[j], father=candidate1), np.where(self.findson_atom(candidate2[j], father=candidate1)!=1)[0]))   
                                 
                                 #아들 일치 여부 확인
                                 tmp_candidate4 = []
                                 for j in range(len(next_step)):
                                     tmp_candidate4.append(sorted(self.findson_atom(next_step[j], father=candidate2)))
                                 if self.correct_atom_num(tmp_candidate4, [[1], [1,1,8], [6]]) == 3 :   #tmp_cand4:5세대 후보 검증중
                                     candidate4 = []
                                     for j in range(len(next_step)):                                        #cand4 할당
                                         tmp = sorted(list(self.findson_index(next_step[j], [i] + candidate1 + candidate2 + candidate3)))
                                         candidate4.extend(tmp)
                                         
                                    #5세대 중 C인 애의 아들이 2중결합 O인가?
                                    #next_step:5세대 중 C인 놈
                                     next_step = self.get_multiple_elements(candidate4, np.where(self.mol_feat[candidate4]==6)[0]) 
                                          
                                     #그놈 아들중 O인놈
                                     if len(next_step)==1:
                                         tmp = self.get_multiple_elements(self.findson_index(next_step[0], father=candidate3), np.where(self.findson_atom(next_step[0], father=candidate3)==8)[0]) 
                                         #과 2중결합으로 연결돼있다면?
                                         if bool(self.mol_adj[next_step,tmp]==2):
                                             candidate5 = tmp
                                                  
                                             coo_ring_list.append(sorted([i]+candidate1+candidate2+candidate3+candidate4+candidate5)) 
                                                 
                                                
        del_list = sum(coo_ring_list, [])       #삭제해야 할 목록들       
        
     #   self.mol_adj[del_list,:] = 0  
      #  self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
        
        return(coo_ring_list)      
    
    def ch2ch2ch2o(self):
        ch2ch2ch2o_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 6 :    #i번째 원소가 C일 경우
                if self.correct_atom_num(self.findson_atom(i), [6,6,1,1]) >=4 :              #i번째 원소의 아들이 cchh일경우
                    candidate1 = self.findson_index(i)
                    next_step = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==6)[0])  #next_step: 2세대중 c인애들
                    
                    #3세대가 [hho],[hh?]인지?
                    tmp = []
                    tmp.append(sorted(list(self.findson_atom(next_step[0], [i]))))
                    tmp.append(sorted(list(self.findson_atom(next_step[1], [i]))))   
                    
                    
                    if (self.correct_atom_num(tmp, [[1,1,8]]) >= 1 &  #and    
                        #3세대중 [hho] 1개 이상 존재
                       self.correct_atom_num(tmp[0], [1,1]) >= 2 &    #and  
                        #3세대 첫째 h 2개 이상
                       self.correct_atom_num(tmp[1], [1,1]) >= 2) :      
                        #3세대 둘째 h 2개 이상       을 만족하는가?
                        tmp[0] + tmp[1]
                        
                        candidate2 = sorted(self.findson_index(next_step[0], [i]) + self.findson_index(next_step[1], [i]))
                        tmp = list(np.where(self.mol_feat[candidate2]==8)[0])          #o 한개
                        tmp = tmp + list(np.where(self.mol_feat[candidate2]==6)[0][:5])      #h 5개
                        candidate2 = tmp
                        
                        ch2ch2ch2o_list.append(sorted([i]+candidate1+candidate2))
          
                        
        del_list = sum(ch2ch2ch2o_list, [])       #삭제해야 할 목록들       
        
      #  self.mol_adj[del_list,:] = 0  
      #  self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
        
        return(ch2ch2ch2o_list)    
    
    def ch2ch2o(self):
        ch2ch2o_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 6 :      #i번째 원소가 C일 경우
                if self.correct_atom_num(self.findson_atom(i), [1,1,6,8]) >=4 :     #2세대가 [hhco]일 경우
                    candidate1 = self.findson_index(i)
                    next_step = self.get_multiple_elements(candidate1, np.where(self.mol_feat[candidate1] == 6)[0])   #next_step : 2세대 중 C인 애
                    
                    if self.correct_atom_num(self.findson_atom(next_step[0], [i]), [1,1]) >=2 :       #next_step의 아들 중 H가 2개 이상
                        candidate2 = self.findson_index(next_step[0], [i])
                        tmp = list(np.where(self.mol_feat[candidate2]==1)[0][:2])    #next_step 아들 중 h 2개인 애 index
                        candidate2 = self.get_multiple_elements(candidate2, tmp)
                        
                        ch2ch2o_list.append(sorted([i] + candidate1 + candidate2))
                    
        del_list = sum(ch2ch2o_list, [])       #삭제해야 할 목록들       
        
      #  self.mol_adj[del_list,:] = 0  
     #   self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
        
        return(ch2ch2o_list)    
    
    def cooh(self):
        cooh_list = []
        
        for i in range(len(mol_adj)):
            if mol_feat[i] == 6 :      #i번째 원소가 C일 경우
                if (len(list(np.where(mol_adj[i,]==1)[0])) >= 1 and
                    len(list(np.where(mol_adj[i,]==2)[0])) == 1 and
                    #i번째 원소가 1중 1개 이상, 2중 1개로 다른원자와 연결되어있을 경우
                    self.correct_atom_num(self.findson_atom(i), [8,8]) ==2):
                    #i번째 원자의 아들이 oo일 경우
                            
                    candidate1 = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==8)[0])   #oo 할당
                    
                    next_step = []                            #3세대 중 1중결합인 H인것
                    for j in range(len(candidate1)):
                        next_step.extend(self.get_multiple_elements(self.findson_index(candidate1[j], [i]), np.where(self.findson_atom(candidate1[j], [i])==1)[0]))   
                            
                    if len(next_step) >=1 :                   #3세대중 H인것 1개 이상 존재할경우
                        candidate2 = [next_step[0]]           #할당
                            
                        cooh_list.append(sorted([i] + candidate1 + candidate2))
                        
        del_list = sum(cooh_list, [])       #삭제해야 할 목록들       
        
      #  self.mol_adj[del_list,:] = 0  
      #  self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
        
        return(cooh_list)    
    
    def ch3(self):
        ch3_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 6 :    #i번째 원소가 C일 경우
                if self.correct_atom_num(self.findson_atom(i), [1,1,1]) >=3:   #c의 아들중 h가 3개 이상일 경우
                    candidate1 = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==1)[0])  #can1: c의 아들중 h인애들
                    #candidate1.extend([i])               #문제없으면 나중에 지우기
                    ch3_list.append(sorted([i] + candidate1)) 
        
        del_list = sum(ch3_list, [])       #삭제해야 할 목록들       
        
     #   self.mol_adj[del_list,:] = 0  
     #   self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
    
        return ch3_list
    
    def coo(self):
        coo_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 6 :      #i번째 원소가 C일 경우
                if (len(list(np.where(self.mol_adj[i,]==1)[0])) >= 1 and
                    len(list(np.where(self.mol_adj[i,]==2)[0])) == 1 and
                    #i번째 원소가 1중 1개, 2중 1개로 다른원자와 연결되어있을 경우
                    self.correct_atom_num(self.findson_atom(i), [8,8]) ==2):
                    #i번째 원자의 아들이 oo일 경우
                    
                    candidate1 = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==8)[0])  
                     
                    coo_list.append(sorted([i] + candidate1))
                    
        del_list = sum(coo_list, [])       #삭제해야될 목록들
        
     #   self.mol_adj[del_list, :] = 0
     #   self.mol_adj[:,del_list] = 0
        self.mol_feat[del_list] = 0
        
        return(coo_list)
    

    
    def ch2(self):
        ch2_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 6 :    #i번째 원소가 C일 경우
                if self.correct_atom_num(self.findson_atom(i), [1,1]) >=2:   #c의 아들중 h가 2개 이상일 경우
                    candidate1 = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==1)[0])  #can1: c의 아들중 h인애들
                    #candidate1.extend([i])         #문제없으면 나중에 지우기
                    ch2_list.append(sorted([i] + candidate1)) 
        del_list = sum(ch2_list, [])       #삭제해야 할 목록들       
        
      #  self.mol_adj[del_list,:] = 0  
      #  self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
        return ch2_list

    def oh(self):
        oh_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 8 : #i번째 원소가 O일 경우
                if self.correct_atom_num(self.findson_atom(i), [1]) >=1 :  #o의 아들중 h가 1개 이상일 경우
                    candidate1 = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==1)[0])  #can1: c의 아들중 h인애들
                    #candidate1.extend([i])            #문제없으면 나중에 지우기
                    oh_list.append(sorted([i] + candidate1)) 
                    
        del_list = sum(oh_list, [])       #삭제해야 할 목록들       
        
     #   self.mol_adj[del_list,:] = 0  
     #   self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
                    
        return oh_list
    
    def ch(self):
        ch_list = []
        
        for i in range(len(self.mol_adj)):
            if self.mol_feat[i] == 6 :    #i번째 원소가 C일 경우
                if self.correct_atom_num(self.findson_atom(i), [1]) >=1:   #c의 아들중 h가 1개 이상일 경우
                    candidate1 = self.get_multiple_elements(self.findson_index(i), np.where(self.findson_atom(i)==1)[0])  #can1: c의 아들중 h인애들
                   # candidate1.extend([i])        #문제없으면 나중에 지우기
                    ch_list.append(sorted([i] + candidate1)) 
                    
        del_list = sum(ch_list, [])       #삭제해야 할 목록들       
        
     #   self.mol_adj[del_list,:] = 0  
     #   self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
                    
        return ch_list
    
    def o(self):
        o_list = []
        tmp = np.where(self.mol_feat == 8)[0].tolist()       #o인 애들
        
        for i in range(len(tmp)):
            if len(np.where(mol_adj[tmp[i],]==2)[0])<=0 :          #2중결합이 존재하지 않는다면
                o_list.append([tmp[i]])
        
        del_list = sum(o_list, [])
        
     #   self.mol_adj[del_list,:] = 0  
     #   self.mol_adj[:,del_list] = 0  
        self.mol_feat[del_list] = 0 
        
        return(o_list)

In [48]:
def scoring():
    #Mol weight
    weight = Descriptors.MolWt(Chem.MolFromSmiles(mol_smiles))

    #partition coefficient
    logP = Descriptors.MolLogP(Chem.MolFromSmiles(mol_smiles))
    logP_score = max( min(   np.log  (  2/  (  abs  ( logP  )  +0.01  )  )    ,1.1 ), 0)
    
    #max distance
    max_distance_matrix = distance_matrix(mol_coord, mol_coord)
    max_distance_index = np.where(max_distance_matrix==np.max(max_distance_matrix))
    
    #hydro/lipo
    hydrophilic = so4na_list + cook_list + coona_list + nr2_list + coo_list + coo_ring_list + cooh_list + oh_list + o_list + ch2ch2o_list   
    lipophilic = ch_list + ch2_list + ch3_list + ch2ch2ch2o_list
    
    ##
    #print(mol_coord.shape)
    #print(len(lipophilic))
    #print(len(hydrophilic))
    ##
    
    #if len(lipophilic)==0 or len(hydrophilic)==0 or len(hydrophilic)==1 or 450>weight or 950<weight or logP>6 or logP<-1:  
    if len(lipophilic)==0 or len(hydrophilic)==0 or len(hydrophilic)==1:      
        #친유가 0개 or 친수가 0 or 1개라면 계산불가능, pass 
        #distance_score=None
        #pass
        return(weight, len(lipophilic), len(hydrophilic),  None, logP, logP_score, None, None, None, None, None, None)
        
    else:         
        #lipo_coord
        
        lipophilic_coord = pd.DataFrame(index = range(len(lipophilic)), columns = range(3))
        
        for i in range(len(lipophilic)):
            lipophilic_coord.iloc[i,0] = np.mean(mol_coord[lipophilic[i],0])
            lipophilic_coord.iloc[i,1] = np.mean(mol_coord[lipophilic[i],1])
            lipophilic_coord.iloc[i,2] = np.mean(mol_coord[lipophilic[i],2])       
            
        #친유성 군집내 거리
        if len(lipophilic)==1:
            lipophilic_dist = np.nan                               #군집이 한개라면 군집내거리 평균은 NA
        else:
            aa = distance_matrix(lipophilic_coord, lipophilic_coord)
            lipophilic_dist = np.mean(aa[np.where(aa!=0)])    #군집내 거리 평균
            
        #hydro_coord 
        hydrophilic_coord = pd.DataFrame(index = range(len(hydrophilic)), columns = range(3))
        
        for i in range(len(hydrophilic)):
            hydrophilic_coord.iloc[i,0] = np.mean(mol_coord[hydrophilic[i],0])
            hydrophilic_coord.iloc[i,1] = np.mean(mol_coord[hydrophilic[i],1])
            hydrophilic_coord.iloc[i,2] = np.mean(mol_coord[hydrophilic[i],2])
        
        ##
       # print(lipophilic_coord.shape)
        #print(hydrophilic_coord.shape)
        ##
        
        #hydro_coord_군집 나누기
        clustering_ensemble = ClusteringEnsemble()
        try:
            last_cluster = clustering_ensemble.ensemble(hydrophilic_coord)
            hydrophilic_coord1 = hydrophilic_coord.iloc[list(last_cluster==0),:]    #친수1번
            hydrophilic_coord2 = hydrophilic_coord.iloc[list(last_cluster==1),:]    #친수2번

            #친수성1 군집내 거리
            if len(hydrophilic_coord1)==1:
                hydrophilic_coord1_dist = np.nan                               #군집이 한개라면 군집내거리 평균은 NA
            else:
                bb = distance_matrix(hydrophilic_coord1, hydrophilic_coord1)
                hydrophilic_coord1_dist = np.mean(bb[np.where(bb!=0)])    #군집내 거리 평균


            #친수성1 군집내 거리
            if len(hydrophilic_coord2)==1:
                hydrophilic_coord2_dist = np.nan                               #군집이 한개라면 군집내거리 평균은 NA
            else:
                cc = distance_matrix(hydrophilic_coord2, hydrophilic_coord2)
                hydrophilic_coord2_dist = np.mean(cc[np.where(cc!=0)])    #군집내 거리 평균

            #친수간 군집 간 거리
            hh_dist = np.linalg.norm(np.mean(hydrophilic_coord1, axis=0)- np.mean(hydrophilic_coord2, axis=0))        
            score1 =  hh_dist / np.max(max_distance_matrix)    #친수군집간거리 / 양 극단간 거리
            #1 - (군집내거리 평균 / 양 극단간 거리)
            df = pd.DataFrame([lipophilic_dist, hydrophilic_coord1_dist, hydrophilic_coord2_dist])
            score2 = 1 - np.mean(np.array(df.dropna()))/np.max(max_distance_matrix)
            #친수성기간 크기 비교
            score3 = max(len(hydrophilic_coord1), len(hydrophilic_coord2)) / min(len(hydrophilic_coord1), len(hydrophilic_coord2))        
            #양극단 원자 친수 여부 
            score4 = sum(hydrophilic, []).count(max_distance_index[0][0]) + sum(hydrophilic, []).count(max_distance_index[0][1])

        #geometrical score
            geometrical_score = (score1*2) + (score2*1) + max((score3*0.5),0.75) + min((score4*1),2)

            total_score = (1.5*geometrical_score) + (0.5 * mol_pps) + (0.2 * logP_score)

            return(weight, len(lipophilic), len(hydrophilic_coord1),  len(hydrophilic_coord2), logP, logP_score, score1, score2, score3, score4, geometrical_score, total_score )

            
        except:
            CLT_ERR_NUMS.append(npc)
            return(weight, len(lipophilic), len(hydrophilic),  None, logP, logP_score, None, None, None, None, None, None)
        
       

In [49]:
result = pd.DataFrame(index = range(len(dataset)), columns = ["so4na", "coona", "cook", "nr2", "coo_ring", "ch2ch2ch2o", "ch2ch2o",
                                                               "cooh", "ch3", "coo",  "ch2", "oh", "ch", "o", "PPS", "weight", 
                                                               "logP", "logP_score", "lipophilic", "hydrophilic1", "hydrophilic2", "score1", "score2",
                                                               "score3", "score4", "geometrical_score", "total_score","Err"])
result = result.reindex(index = dataset.loc[:,'mdb_id'])

In [11]:
smiles_to_input = SmilesToInput(smiles_input_dic)
adj,feat,coord,smiles,pps = smiles_to_input.make_dic()

0 28615


/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/dgl/base.py:25: UserWarning: Currently adjacency_matrix() returns a matrix with destination as rows by default.  In 0.5 the result will have source as rows (i.e. transpose=True)
  warnings.warn(msg, warn_type)


2 28617
4 28619
5 28620
9 28624
11 28626
94 28709
119 28734
183 28798
218 28833
256 28871
351 28966
402 29017
412 29027
524 29139
639 29254
834 29449
1136 29751
1140 29755
1450 30065
1494 30109
1692 30307
1761 30376
1866 30481
1899 30514
1939 30554
2046 30661
2297 30912
2301 30916
2385 31000
2482 31097
2601 31216
2617 31232
2887 31502
2992 31607
3023 31638
3093 31708
3096 31711
3102 31717
3155 31770
3264 31879
3332 31947
3564 32179
3721 32336
3775 32390
3797 32412
3847 32462
3977 32592
4020 32635
4038 32653
4263 32878
4455 33070
4479 33094
4488 33103
4489 33104
4514 33129
4524 33139
4528 33143
4605 33220
4610 33225
4628 33243
4638 33253
4653 33268
4662 33277
4675 33290
4677 33292
4731 33346
4772 33387
4855 33470
4907 33522
4930 33545
4932 33547
5004 33619
5027 33642
5055 33670
5126 33741
5128 33743
5131 33746
5160 33775
6565 35180
7112 35727
7268 35883
7304 35919
7436 36051
7507 36122
7513 36128
7588 36203
7636 36251
7680 36295
7698 36313
7720 36335
7873 36488
7938 36553
7981 36596
802

In [12]:
too_much_list , idx = ion_too_much(adj,feat)

In [13]:
oh_ring_list, idx = oh_ring(adj,feat,idx)

In [50]:
###########진짜
for npc in idx:
    
    mol_adj = copy.deepcopy(adj[npc])
    mol_feat = copy.deepcopy(feat[npc])
    mol_coord = copy.deepcopy(coord[npc])
    mol_smiles = copy.deepcopy(smiles[npc])
    mol_pps = copy.deepcopy(pps[npc])
    
    cal = calculating(mol_adj, mol_feat)
    
    #index 추출하여 저장
    so4na_list = cal.so4na()
    coona_list = cal.coona()
    cook_list = cal.cook()
    coo_ring_list = cal.coo_ring()
    ch2ch2ch2o_list = cal.ch2ch2ch2o()
    ch2ch2o_list = cal.ch2ch2o()
    cooh_list = cal.cooh()
    ch3_list = cal.ch3()
    coo_list = cal.coo()
    ch2_list = cal.ch2()
    oh_list = cal.oh()
    ch_list = cal.ch()
    o_list = cal.o()    
    nr2_list = cal.nr2()
    
    weight, lipophilic, hydrophilic1, hydrophilic2, logP, logP_score, score1, score2, score3, score4, geometrical_score, total_score = scoring() 
    
    result.loc[npc,"so4na"] = len(so4na_list)
    result.loc[npc,"coona"] = len(coona_list)
    result.loc[npc,"cook"] = len(cook_list)        
    result.loc[npc,"nr2"] = len(nr2_list)
    result.loc[npc,"coo_ring"] = len(coo_ring_list)
    result.loc[npc,"ch2ch2ch2o"] = len(ch2ch2ch2o_list)
    result.loc[npc,"ch2ch2o"] = len(ch2ch2o_list)
    result.loc[npc,"cooh"] = len(cooh_list)
    result.loc[npc,"ch3"] = len(ch3_list)
    result.loc[npc,"coo"] = len(coo_list)    
    result.loc[npc,"ch2"] = len(ch2_list)
    result.loc[npc,"oh"] = len(oh_list)
    result.loc[npc,"ch"] = len(ch_list)
    result.loc[npc,"o"] = len(o_list)
    result.loc[npc,"PPS"] = mol_pps
    result.loc[npc,"weight"] = weight
    result.loc[npc,"logP"] = logP
    result.loc[npc,"logP_score"] = logP_score
    result.loc[npc,"lipophilic"] = lipophilic
    result.loc[npc,"hydrophilic1"] = hydrophilic1
    result.loc[npc,"hydrophilic2"] = hydrophilic2
    result.loc[npc,"score1"] = score1
    result.loc[npc,"score2"] = score2
    result.loc[npc,"score3"] = score3
    result.loc[npc,"score4"] = score4
    result.loc[npc,"geometrical_score"] = geometrical_score
    result.loc[npc,"total_score"] = total_score   
    
    if 450>weight or 950<weight:
            result.loc[npc,"Err"] = "unsuitable weight" 
            
    if logP>6 or logP<-1:
            result.loc[npc,"Err"] = "unsuitable logP"
            
    if hydrophilic2==None: 
            result.loc[npc,"Err"] = "Do not have clusters"
            
    print(npc)

28616
28618
28621
28622
28623
28625
28627
28628
28629
28630


/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1590: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpac

28631
28632
28633
28634


/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/jjh/ana

28635
28636
28637


...

/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1590: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1590: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now construct

39234
39235
39236
39237
39238
39239
39240
39241


/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1590: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1590: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1590: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1590: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square ma

39242
39243
39244


/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/jjh/anaconda3/envs/ddsai/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"
/home/jjh/ana

In [51]:
result.loc[ERR_NUMS,'Err'] = "conformation Error"
result.loc[oh_ring_list,'Err'] = "This molcule is OH_ring"
result.loc[too_much_list,'Err'] = "This molecule has more than two ions."
result.loc[CLT_ERR_NUMS,'Err'] = "clustering problem."

In [52]:
result.to_csv("/BiO1/projects/tmp/result201202_1.csv")